In [55]:
import pandas as pd
import datetime
import time
import unicodedata
import string
import requests
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

import pandas as pd
import datetime

import os
# .env ファイルをロードして環境変数へ反映
from dotenv import load_dotenv
load_dotenv('.env')
# 環境変数を参照

def removal_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text

def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

def post_line_text_and_image(message,image_path,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files) 

def insert_data_bulk(df,cnx):
    insert_sql = f"""INSERT INTO {os.getenv('WORDPRESS_DB_TABLE')}(店舗名, 日付, Nのつく日, 都道府県, 機種名, 台番号, G数, 差枚, BB, RB,ART, BB確率, RB確率, ART確率, 合成確率) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    cur = cnx.cursor()
    cur.executemany(insert_sql, df.values.tolist())
    cnx.commit()
    print("Insert bulk data")

def delete_data(cnx,day):
    cursor = cnx.cursor()
    target_days_ago = datetime.date.today() - datetime.timedelta(days=day)
    target_days_ago_str = target_days_ago.strftime('%Y-%m-%d')
    target_days_ago_str
    sql = f"DELETE FROM {os.getenv('WORDPRESS_DB_TABLE')} WHERE 日付 < '{target_days_ago_str} 00:00:00';"
    cursor.execute(sql)
    cnx.commit()

prefecture_list = ['鳥取県']#,'東京都''静岡県','岐阜県','愛知県','三重県'
line_token = os.getenv('LINE_TOKEN')
#cols = ['機種名', '台番号', 'G数', '差枚', 'BB', 'RB', 'ART', 'BB確率', 'RB確率', 'ART確率','合成確率','店舗名']
ichiran_all_tennpo_df = pd.DataFrame(index=[], columns=[])
#print(line_token)


url = f'https://nanaasu.com/research/'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')



block = soup.find('div',class_='research-block')
#print(block.prettify())
pledge_df = pd.DataFrame(index=[], columns=['media_name','syuzai_name','pledge_text'])
ul_list = block.find_all('ul')
for i,ul in enumerate(ul_list[1:-14]):
    try:
        li_list = ul.find_all('li')
        media_name = str(li_list).split('<br/>')[0].replace('[<li>','')
        #print(media_name)
        syuzai_name = ''
        for t in str(li_list).split('<br/>')[1].split('</li>')[0]:
            syuzai_name += t + ' '
        syuzai_name = syuzai_name.replace(' ','')
        #print(syuzai_name)
        pledge_text =  str(li_list).split(' <li>')[1].split('<br/>')[0].replace('</li>]','')
        #print(pledge_text)
        record = pd.DataFrame([[media_name,syuzai_name,pledge_text]], columns=['media_name','syuzai_name','pledge_text'])
        pledge_df = pd.concat([pledge_df, record],axis=0)
    except Exception as e:
        print(i,'error',e,str(li_list))
        continue
    #break




pledge_df

85 error list index out of range [<li class="red-line-research"></li>]
121 error list index out of range [<li class="red-line-research"></li>]
125 error list index out of range [<li class="red-line-research"></li>]
130 error list index out of range [<li class="red-line-research"></li>]
146 error list index out of range [<li class="red-line-research"></li>]
153 error list index out of range [<li class="red-line-research"></li>]
157 error list index out of range [<li class="red-line-research"></li>]
172 error list index out of range [<li class="red-line-research"></li>]
184 error list index out of range [<li class="red-line-research"></li>]
186 error list index out of range [<li class="red-line-research"></li>]
208 error list index out of range [<li class="red-line-research"></li>]
258 error list index out of range [<li class="red-line-research"></li>]
261 error list index out of range [<li class="red-line-research"></li>]
272 error list index out of range [<li class="red-line-research">

,media_name,syuzai_name,pledge_text
0,アツ姫,満点STARKewlスタークール,3台以上連番3箇所以上が機械割100%以上
0,アツ姫,満天STARZealスタージール,3台以上2機種以上が機械割100%以上
0,アツ姫,満天STARJoyスタージョイ（ジャグラー対象）,ジャグラー3機種以上が機械割100%以上
0,アツ姫,満天STARVigorスタービゴア,バラエティーコーナー対象。機械割100%以上
0,アツ姫,GPLギガプレミアムレジェンド,構成3台以上3機種以上に全456+総台数10％に6+設置3台以上の1機種に全6+5台並び3カ...
...,...,...,...
0,肉はこう焼く！（エースプロ）,パチンコ 肉はこう焼く！タン,角台系
0,肉はこう焼く！（エースプロ）,パチンコ 肉はこう焼く！イチボ,1機種ボーダー以上
0,アナコンダ,アナコンダ,ハナハナシリーズ1/3が5.6
0,回胴連（KAIDOREN かいどうれん）,回胴連トライデント,設置3台以上機種の3機種以上全5.6


In [53]:
str(li_list)

'[<li>回胴連（KAIDOREN かいどうれん）<br/>回胴連トライデント連</li>, <li>3台以上並びで3カ所以上5.6</li>]'

In [45]:
pledge_df.value_counts('syuzai_name')

syuzai_name
                  451
パチスロ　肉はこう焼く！        7
パチンコ　肉はこう焼く！        4
REAL                2
満天STAR              2
                 ... 
NEW GENERATION      1
Niziyou◎！           1
OUROBOROS           1
P-1                 1
鬼滅女子                1
Name: count, Length: 159, dtype: int64

In [44]:
ul_list[84:87]

[<ul id="atsuhime94">
 <li>アツ姫<br/>平日REVOLUTION</li>
 <li>詳細は調査中</li>
 </ul>,
 <ul id="atsuhime95">
 <li>アツ姫<br/>武勇伝NEO</li>
 <li>設置台数の11%が5.6<br/>
 ※5.6は均等配分</li>
 </ul>,
 <ul>
 <li class="red-line-research"></li>
 </ul>]

In [38]:
syuzai_name = ''
for t in str(li_list).split('<br/>')[1].split('</li>')[0].split(' ')[:-1]:
    syuzai_name += t + ' '
syuzai_name = syuzai_name.rstrip(' ')
print(syuzai_name)

満点STAR Kewl


In [22]:
str(li_list)

'[<li>アツ姫<br/>満点STAR Kewl スタークール</li>, <li>3台以上連番3箇所以上が機械割100%以上<br/>\n            もしくは<br/>\n            3台以上並びで4カ所以上高設定\n            </li>]'